# BM Praktikum 
## DICOM Reader

In [52]:
import vtk

In [53]:
reader = vtk.vtkDICOMImageReader() 
reader.SetDirectoryName('Marching_Man')
reader.Update()

Berechnet das Zentrum des Volumens

In [54]:
xMin, xMax, yMin, yMax, zMin, zMax = reader.GetDataExtent()
xSpacing, ySpacing, zSpacing = reader.GetOutput().GetSpacing()
print(xSpacing, ySpacing, zSpacing)
x0, y0, z0 = reader.GetOutput().GetOrigin()
center = [x0 + xSpacing * 0.5 * (xMin + xMax),
          y0 + ySpacing * 0.5 * (yMin + yMax),
          z0 + zSpacing * 0.5 * (zMin + zMax)]

0.9375 0.9375 1.5


Matrix der Axialen-Sicht auf den Datensatz

In [55]:
axial = vtk.vtkMatrix4x4()
axial.DeepCopy((1, 0, 0, center[0],
                0, 1, 0, center[1],
                0, 0, 1, center[2],
                0, 0, 0, 1))

Extrahieren einer Schicht (slice) in der gewünschten Ausrichtung

In [56]:
reslice = vtk.vtkImageReslice()
reslice.SetInputConnection(reader.GetOutputPort())
reslice.SetOutputDimensionality(2)
reslice.SetResliceAxes(axial)
reslice.SetInterpolationModeToCubic()

Erstellen einer Graustufen-Lookup-Tabelle

In [57]:
glob_level = 0.
glob_window = 2048.
table = vtk.vtkLookupTable()
table.SetRange(glob_level-glob_window/2, glob_level+glob_window/2) # Level-Window -> intensity range
table.SetValueRange(0.0, 1.0) # von schwarz nach weiß
table.SetSaturationRange(0.0, 0.0) # keine Farbsättigung
table.SetRampToLinear() # Linearer Verlauf
table.Build()

Map Bild zu lookup table

In [58]:
color = vtk.vtkImageMapToColors()
color.SetLookupTable(table)
color.SetInputConnection(reslice.GetOutputPort())

Das Bild anzeigen

In [59]:
actor = vtk.vtkImageActor()
actor.GetMapper().SetInputConnection(color.GetOutputPort())

renderer = vtk.vtkRenderer()
renderer.AddActor(actor)

In [60]:
window = vtk.vtkRenderWindow()
window.AddRenderer(renderer)

Einrichten der Interaktion

In [61]:
interactorStyle = vtk.vtkInteractorStyleImage()
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetInteractorStyle(interactorStyle)
window.SetInteractor(interactor)
window.Render()

Definieren von callbacks für das Slicing 

In [62]:
actions = {}
actions["Slicing"] = 0

def ButtonCallback(obj, event):
    if event == "LeftButtonPressEvent":
        actions["Slicing"] = 1
    else:
        actions["Slicing"] = 0

def MouseMoveCallback(obj, event):

    (lastX, lastY) = interactor.GetLastEventPosition()
    (mouseX, mouseY) = interactor.GetEventPosition()
    if actions["Slicing"] == 1:
        deltaY = mouseY - lastY
        reslice.Update()
        sliceSpacing = reslice.GetOutput().GetSpacing()[2]
        matrix = reslice.GetResliceAxes()
# move the center point that we are slicing through
        center = matrix.MultiplyPoint((0, 0, sliceSpacing*deltaY, 1))
        matrix.SetElement(0, 3, center[0])
        matrix.SetElement(1, 3, center[1])
        matrix.SetElement(2, 3, center[2])
        window.Render()
    else:
        interactorStyle.OnMouseMove()

interactorStyle.AddObserver("MouseMoveEvent", MouseMoveCallback)
interactorStyle.AddObserver("LeftButtonPressEvent", ButtonCallback)
interactorStyle.AddObserver("LeftButtonReleaseEvent", ButtonCallback)

6

In [63]:
interactor.Start() # Interaktion starten